In [3]:
import pandas as pd   
import numpy as np

data_X = pd.read_csv('./r_train_X.csv', encoding = 'big5')  #r_train_X.csv文件要与python代码文件(.py或.ipynb)在同一个目录下才行
data_y = pd.read_csv('./r_train_y.csv', encoding = 'big5')  #r_train_y.csv文件要与python代码文件(.py或.ipynb)在同一个目录下才行
X = data_X.to_numpy()  #将数据转换成numpy数据
y = data_y.to_numpy()  #将数据转换成numpy数据

In [ ]:
#函数功能：预处理数据，使所有维度的数据都缩放到零均值，标准差为1
#    输入：X 待处理数据，每一列为同一个属性在不同样本上的取值
#    输出：X_processed 缩放后的数据，维度和 X 一样
#          mean_X 每一列的均值，维度为 (1, n)
#          std_X 每一列的标准差，维度为 (1, n)
def preprocessData(X):
    mean_X = np.mean(X, axis = 0, keepdims = 1)
    std_X = np.std(X, axis = 0, keepdims = 1, ddof=1)
    X_processed = (X - mean_X) / (std_X + 0.00000001)
    return X_processed, mean_X, std_X

In [ ]:
#函数功能：在数据前增加一列或一行全为1
#    输入：X_train 原始数据 (m, n)
#          ax = 1表示在最左边加1列， = 0 表示在最上面加一行
#    输出：X 加完后的数组  (m + 1, n) 或 (m, n + 1)
def expendOneCol(X_train, ax = 1):
    X = np.insert(X_train, 0, values = np.ones((1,X_train.shape[(ax+1)%2])), axis = ax)
    return X;

In [ ]:
#函数功能：初始化参数
#    输入：m 维度
#    输出：parameters 初始化为 0 的数组 (m, 1)
def initialParameters(m):
    parameters = np.zeros((m, 1))
    return parameters

In [ ]:
# 函数功能：计算假设函数的结果，公式如上
#     输入：X (m, n) 是一个 m 行 n 列的矩阵，m 是样本个数，n 是 theta 的维度
#           parameters 是一个 (n, 1) 的 n 维向量，其就是公式中的 theta
#     输出：y (m, 1) 是一个 m 维的列向量，每一行是一个样本的输出
def hypothese(X, parameters):
    h = np.dot(X, parameters)
    return h

In [ ]:
#函数功能：计算梯度
#    输入：X_train 训练数据
#          y 标签数据
#          h 假设函数结果
#    输出：grad 梯度
def calculateGrad(X_train, y, h):
    grad = np.dot(X_train.T, h - y)
    return grad

In [ ]:
#函数功能：计算损失值
#    输入：y 标签
#          h 假设函数结果
#    输出：J 损失值
def calLossFunction(y, h):
    sub = (h - y)
    J = np.dot(sub.T, sub) / 2
    return J

In [ ]:
def linearRegressionTrain(X_train, y, learningRate = 0.000002, iteration = 1000):
    X_processed, mean_X, std_X = preprocessData(X)
    X_train = expendOneCol(X_processed)
    (m, n) = X_train.shape
    parameters = initialParameters(n)
    lossHistory = np.zeros((iteration, 1))
    
    for ite in range(iteration):
        h = hypothese(X_train, parameters)
        grads = calculateGrad(X_train, y, h)
        parameters = parameters - learningRate * grads
        lossHistory[ite] = calLossFunction(y, h)
    
    return lossHistory

In [ ]:
his = linearRegressionTrain(X, y)

In [ ]:
import matplotlib.pyplot as plt
itera = np.arange(1000)
plt.plot(itera, his)
#plt.plot(itera, loss_val_history)  #画出在验证集上损失值变化曲线
plt.show

In [21]:
import numpy as np
class LinearRegressionModel:
    def __init__(self, X, y):
        """
        X_train       用于存储训练集数据
        y_train       用于存储训练数据的标签
        X_mean        用于存储各个属性的均值
        X_std         用于存储各个属性的标准差
        m             用于存储样本个数
        n             用于存储属性个数（扩充后的）
        parameters    用于存储参数数据
        seed          用于存储随机数种子
        method        用于存储训练方法，默认为 None 表示未设定
        iteration     用于存储迭代次数
        miniBatchSize 用于记录 miniBatch 的尺寸，默认为 64
        recoedTimes   用于存储记录损失值的次数
        lossHistory   用于存储记录的损失值
        """
        (temp_X, self.X_mean, self.X_std) = self.preprocessData(X)
        self.X_train = self.expendOneCol(temp_X)
        self.y_train = y       
        self.m, self.n = self.X_train.shape 
        self.parameters = self.initialParameters(self.n) 
        self.seed = 0          
        self.method = "None"   
        self.iteration = 0
        self.miniBatchSize = 64
        self.recoedTimes = 0  
        self.lossHistory = 0   
        
    def preprocessData(self, X):
        """
        函数功能：预处理数据，使所有维度的数据都缩放到零均值，标准差为1
            输入：X 待处理数据，每一列为同一个属性在不同样本上的取值
            输出：X_processed 缩放后的数据，维度和 X 一样
                  mean_X 每一列的均值，维度为 (1, n)
                  std_X 每一列的标准差，维度为 (1, n)
        """     
        X_mean = np.mean(X, axis = 0, keepdims = 1)
        X_std = np.std(X, axis = 0, keepdims = 1, ddof=1)
        X_processed = (X - X_mean) / (X_std + 0.00000001)
        return X_processed, X_mean, X_std
    
    def expendOneCol(self, X_train, ax = 1):
        """
        函数功能：计算假设函数的结果，公式如上
            输入：X (m, n) 是一个 m 行 n 列的矩阵，m 是样本个数，n 是 theta 的维度
                  parameters 是一个 (n, 1) 的 n 维向量，其就是公式中的 theta
            输出：y (m, 1) 是一个 m 维的列向量，每一行是一个样本的输出
        """
        X = np.insert(X_train, 0, values = np.ones((1,X_train.shape[(ax+1)%2])), axis = ax)
        return X;
    
    def initialParameters(self, m):
        """
        函数功能：初始化参数
            输入：m 维度
            输出：parameters 初始化为 0 的数组 (m, 1)"""
        parameters = np.zeros((m, 1))
        return parameters
    
    def hypothese(X, parameters):
        """
        函数功能：计算假设函数的结果，公式如上
            输入：X (m, n) 是一个 m 行 n 列的矩阵，m 是样本个数，n 是 theta 的维度
                  parameters 是一个 (n, 1) 的 n 维向量，其就是公式中的 theta
            输出：y (m, 1) 是一个 m 维的列向量，每一行是一个样本的输出
        """
        h = np.dot(X, parameters)
        return h
    
    def calculateGrad(X_train, y, h):
        """
        函数功能：计算梯度
            输入：X_train 训练数据
                  y 标签数据
                  h 假设函数结果
            输出：grad 梯度
        """
        grad = np.dot(X_train.T, h - y)
        return grad
    
    def calLossFunction(y, h):
        """
        函数功能：计算损失值
            输入：y 标签
                  h 假设函数结果
            输出：J 损失值
        """
        sub = (h - y)
        J = np.dot(sub.T, sub) / 2
        return J

In [22]:
model = LinearRegressionModel(X, y)

In [23]:
model.X_mean

array([[2.25380287e+01, 2.25447531e+01, 2.25513007e+01, 2.25553707e+01,
        2.25578482e+01, 2.25580959e+01, 2.25581136e+01, 2.25571226e+01,
        2.25530526e+01, 1.70237126e+00, 1.70228278e+00, 1.70215891e+00,
        1.70212352e+00, 1.70199965e+00, 1.70194656e+00, 1.70187577e+00,
        1.70184038e+00, 1.70176960e+00, 3.89012564e-01, 3.88931163e-01,
        3.88959476e-01, 3.89170058e-01, 3.89341709e-01, 3.89385949e-01,
        3.89308087e-01, 3.89056804e-01, 3.88511768e-01, 1.40143337e-01,
        1.40185808e-01, 1.40440630e-01, 1.40799858e-01, 1.41019289e-01,
        1.41100690e-01, 1.41134313e-01, 1.41097151e-01, 1.40895417e-01,
        2.14765528e+00, 2.15061051e+00, 2.15280481e+00, 2.15515838e+00,
        2.15565387e+00, 2.15591931e+00, 2.15558308e+00, 2.15372500e+00,
        2.14777915e+00, 1.01154486e+01, 1.01190232e+01, 1.01287206e+01,
        1.01442577e+01, 1.01574058e+01, 1.01650858e+01, 1.01686073e+01,
        1.01672978e+01, 1.01591754e+01, 1.22488055e+01, 1.225514